# Data Wrangling: Join, Combine, and Reshape

In [ ]:
#! ipython suppress id=a3ea397f76e34983a5af4ea3f35454d5
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

- Data may be spread across a number of files or databases or be arranged in a form that is not convenient to analyze. 

- Chapter focuses on :
  
  -  Hierarchical Indexing
  
  -  Data manipulations

## Hierarchical Indexing

- Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis.

In [ ]:
data1 = pd.Series(np.random.randn(9), index=[1, 2, 3, 1, 3, 1, 2, 2, 3] )
data1

1   -0.204708
2    0.478943
3   -0.519439
1   -0.555730
3    1.965781
1    1.393406
2    0.092908
2    0.281746
3    0.769023
dtype: float64

In [ ]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

This is a Series with a MultiIndex as its index. The “gaps” in the index display mean “use the label directly above”:

In [ ]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, partial indexing is possible, enabling you to concisely select subsets of the data:

In [ ]:
data['b']

1    0.274992
3    0.228913
dtype: float64

In [ ]:
data['b':'c']

b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
dtype: float64

In [ ]:
data.loc[["b", "c"]]

b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
dtype: float64

- Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. 

- For example, you can rearrange this data into a DataFrame using its unstack method

In [ ]:
data

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

In [ ]:
data.unstack()

,1,2,3
a,1.246435,1.007189,-1.296221
b,0.274992,NaN,0.228913
c,1.352917,0.886429,NaN
d,NaN,-2.001637,-0.371843


The inverse operation of unstack is stack:

In [ ]:
data.unstack().stack()

a  1    1.246435
   2    1.007189
   3   -1.296221
b  1    0.274992
   3    0.228913
c  1    1.352917
   2    0.886429
d  2   -2.001637
   3   -0.371843
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [ ]:
pd.DataFrame(np.arange(12).reshape((4, 3)), 
            index=[['a', 'a', 'b', 'b'],[1, 2, 1, 2]],  
            columns=[['Ohio', 'Ohio', 'Colorado']])

Ohio Ohio Colorado
a 1    0    1        2
  2    3    4        5
b 1    6    7        8
  2    9   10       11

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [ ]:
frame.index.names = ['key1', 'key2']
frame

Ohio     Colorado
          Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

You can see how many levels an index has by accessing its nlevels attribute:

In [ ]:
frame.index.nlevels

2

With partial column indexing you can similarly select groups of columns:

In [ ]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reordering and Sorting Levels

- At times you may need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. 

- The **swaplevel** method takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [ ]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### Index Sorting

- **sort_index** by default sorts the data lexicographically using all the index levels.


In [ ]:
df = pd.DataFrame([1, 2, 3, 4, 5], index=[100, 29, 234, 1, 150],
                   columns=['A'])

df

,A
100,1
29,2
234,3
1,4
150,5


In [ ]:
df.sort_index()


,A
1,4
29,2
100,1
150,5
234,3


In [ ]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

sort_index by default sorts the data lexicographically using all the index levels

In [ ]:
frame.sort_index()

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11


- You can choose to only use a single level or a subset of levels to sort by passing the level argument. For example::

In [ ]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

### Summary Statistics by Level

- Many descriptive and summary statistics on DataFrame and Series have a level option in which you can specify the level you want to aggregate by on a particular axis. 
  
- Consider the DataFrame below; we can aggregate by level on either the rows or columns:

In [ ]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [ ]:
frame.sum(level='key2')

/var/folders/1h/b7ng0kgj3w78mg7n8k7q7rch0000gn/T/ipykernel_96874/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [ ]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [ ]:
frame.sum(level='color', axis=1)

/var/folders/1h/b7ng0kgj3w78mg7n8k7q7rch0000gn/T/ipykernel_96874/4133796543.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

- You may want to use one or more columns from a DataFrame as the row index or you may wish to move the row index into the DataFrame’s columns. 
 
- Here’s an example DataFrame:

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s set_index function will create a new DataFrame using one or more of its columns as the index:

In [ ]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default the columns are removed from the DataFrame, though you can leave them in by passing drop=False to set_index:

In [ ]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index, on the other hand, does the opposite of set_index; the hierarchical index levels are moved into the columns:

In [ ]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:


  
- Pandas.merge connects rows in DataFrames based on one or more keys. This is similar to SQL or other relational databases, as it implements database join operations.
  
- Pandas.concat concatenates or "stacks" together objects along an axis.
  
- The combine_first instance method enables splicing together overlapping data to fill in missing values in one object with values from another.
  


### Database-Style DataFrame Joins

> Merge or join operations combine datasets by linking rows using one or more keys. 

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [ ]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [ ]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


#### Many-to-one

Below is an example of many-to-one join; the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column. 

In [ ]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [ ]:
df1.merge(df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


> You may notice that the "c" and "d" values and associated data are missing from the result.

> By default pandas.merge does an "inner" join; the keys in the result are the intersection, or the common set found in both tables.


![](./images/pandasjoin.png)



> If we do not specify which column to join on. Pandas.merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [ ]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them separately:

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})


In [ ]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [ ]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


If the column names are different in each object, you can specify them separately:

In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0



-  The outer join takes the union of the keys, combining the effect of applying both left and right joins:

#### Examples

![](./images/pandasjoin.png)

In [ ]:
pd.merge(df1, df2) # by default, it is inner join or sometimes called natural join.

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [ ]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [ ]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


> In an outer join, rows from the left or right DataFrame objects that do not match on keys in the other DataFrame will appear with NA values in the other DataFrame's columns for the non-matching rows.

### Many-to-many 

- Many-to-many merges form the Cartesian product of the matching keys. Here’s an example:

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})


In [ ]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [ ]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


![](./images/pandasjoin.png)

In [ ]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [ ]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


To merge with multiple keys, pass a list of column names:

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})


In [ ]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [ ]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [ ]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


A last issue to consider in merge operations is the treatment of overlapping column names. For example:

In [ ]:
pd.merge(left, right, on='key1')


,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


pandas.merge has a suffixes option for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

- In some cases, the merge key(s) in a DataFrame will be found in its index (row labels). 
  
- In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key:

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [ ]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [ ]:
right1

,group_val
a,3.5
b,7.0


In [ ]:
df_answer =  pd.merge(left1, right1, left_on='key', right_index=True)
df_answer

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [ ]:
df_answer.to_csv('df_answer.csv')

> you will notice that the index values for left1 have been preserved, whereas in other examples above, the indexes of the input DataFrame objects are dropped. Because the index of right1 is unique, this "many-to-one" merge (with the default how="inner" method) can perserve the index values from left1 that correspond to rows in the output

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Using the indexes of both sides of the merge is also possible:



In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [ ]:
left2


,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [ ]:
right2


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### Join

> Join columns of another DataFrame. Join columns with other DataFrame either on index or on a key column. Efficiently join multiple DataFrame objects by index at once by passing a list.




- DataFrame has a join instance method to simplify merging by index. 

- It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns. 
  


In [ ]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [ ]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [ ]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Compare previous example using merge(), we can see Join simplify merging by index

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


> Compared with **pandas.merge**, DataFrame’s **join** method performs a left join on the join keys by default. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:



In [ ]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [ ]:
right1

,group_val
a,3.5
b,7.0


In [ ]:
left1.join(right1, on='key') # how = left by default

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
left1.join(right1, on='key', how = "left") # how = left by default

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [ ]:
left1.join(right1, on='key', how = "inner")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


> For simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general pandas.concat function described in the next section:



In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [ ]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [ ]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [ ]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [ ]:
left2.join([right2, another])


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [ ]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

- Another kind of data combination operation is referred to interchangeably as concatenation or stacking. 

#### Concatenating along axis using Numpy

> **Concatenate** Join a sequence of arrays along an existing axis.



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [ ]:
np.concatenate([arr, arr]) # concatenate along axis 0 by default

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [ ]:
np.concatenate([arr, arr], axis = 1) # concatenate along axis 0 by default

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

### Concatenating along Axis using Pandas

- Pandas objects such as Series and DataFrame, have labeled axes and enable you to further generalize array concatenation.
  
  - If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the values in common?
  
  - Do the concatenated chunks of data need to be identifiable as such in the resulting object?
  
  - Does the "concatenation axis" contain data that needs to be preserved? In many cases, the default integer labels in a DataFrame are best discarded during concatenation.




> The concat function in pandas provides a consistent way to address each of these questions.

Suppose we have three Series with no index overlap:

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [ ]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [ ]:
s3 = pd.Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

Calling pandas.concat with these objects in a list glues together the values and indexes:



In [ ]:
pd.concat([s1, s2, s3]) # concatenate along axis 0 by default

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

> By default pandas.concat works along axis="index", producing another Series. If you pass axis="columns", the result will instead be a DataFrame:



In [ ]:
# pd.concat([s1, s2, s3], axis=1) # same as axis = "columns"
pd.concat([s1, s2, s3], axis= "columns") # same as axis = "columns"

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


>Default join is Outer

In this case there is no overlap on the other axis, which as you can see is the union (the "outer" join) of the indexes. You can instead intersect them by passing join="inner":



You can instead intersect them by passing join="inner"

In [ ]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [ ]:
pd.concat([s1, s4], axis="columns")


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [ ]:
pd.concat([s1, s2], axis=1) # outer is default

,0,1
a,0.0,NaN
b,1.0,NaN
c,NaN,2.0
d,NaN,3.0
e,NaN,4.0


In [ ]:
pd.concat([s1, s4], axis=1, join='inner') # inner or outer

,0,1
a,0,0
b,1,1


Sometimes, in DataFrames, the row index does not contain any relevant data:

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [ ]:
df1

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810


In [ ]:
df2

,b,d,a
0,-0.713544,-0.831154,-2.370232
1,-1.860761,-0.860757,0.560145


In [ ]:
pd.concat([df1, df2])

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
0,-2.370232,-0.713544,NaN,-0.831154
1,0.560145,-1.860761,NaN,-0.860757


In [ ]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
3,-2.370232,-0.713544,NaN,-0.831154
4,0.560145,-1.860761,NaN,-0.860757


## So far ... 

> Concat function concatenates dataframes along rows or columns. We can think of it as stacking up multiple dataframes.



![](./images/concatenate.png)

> Merge: combines dataframes based on values in shared columns. Merge function offers more flexibility compared to concat function because it allows combinations based on a condition.

![](./images/merged.png)

> A very high level difference is that merge() is used to combine two (or more) dataframes on the basis of values of common columns (indices can also be used, use left_index=True and/or right_index=True), 

> Concat() is used to append one (or more) dataframes one below the other (or sideways, depending on whether the axis option is set to 0 or 1).

>  join() is used to merge 2 dataframes on the basis of the index; instead of using merge() with the option left_index=True we can use join().



### Combining Data with Overlap

- There is another data combination situation that can’t be expressed as either a merge or concatenation operation. 

- You may have two datasets whose indexes overlap in full or part.



In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [ ]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [ ]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

> "combine_first", update null elements with value in the same location in 'other'.

In [ ]:
#combine two Series objects by filling null values in one Series with non-null values from the other Series. 
# Result index will be the union of the two indexes.

a.combine_first(b) # fill null values in a with non-null values from b

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    5.0
dtype: float64

With DataFrames, **combine_first** does the same thing column by column, so you can think of it as “patching” missing data in the calling object with data from the object you pass:



In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [ ]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [ ]:
df2


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [ ]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


> The output of combine_first with DataFrame objects will have the union of all the column names.



## Reshaping and Pivoting

- There are a number of basic operations for rearranging tabular data.

- These are alternatingly referred to as reshape or pivot operations.

### Reshaping with Hierarchical Indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:



**Stack**: This “rotates” or pivots from the columns in the data to the rows

**unstack** : This pivots from the rows into the columns


![](2022-09-30-20-13-31.png)

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Using the stack method on this data pivots the columns into the rows, producing a Series:



In [ ]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

From a hierarchically indexed Series, you can rearrange the data back into a DataFrame with unstack:



In [ ]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


> By default the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:



In [ ]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [ ]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


> By default, the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:



In [ ]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [ ]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [ ]:
result.unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


### Pivoting “Long” to “Wide” Format

- A common way to store multiple time series in databases and CSV files is what is sometimes called long or stacked format. 

- In this format, individual values are represented by a single row in table rather than multiple values per row.

> Pivot

> Melt 

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


> wide_to_long()

In [ ]:
import numpy as np 
np.random.seed(123)
df = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                    "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                    "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                    "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                    "X"     : dict(zip(range(3), np.random.randn(3)))})
df["id"] = df.index
df

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-1.085631,0
1,b,e,1.2,1.3,0.997345,1
2,c,f,0.7,0.1,0.282978,2


In [ ]:
pd.wide_to_long(df, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-1.085631,a,2.5
1,1970,0.997345,b,1.2
2,1970,0.282978,c,0.7
0,1980,-1.085631,d,3.2
1,1980,0.997345,e,1.3
2,1980,0.282978,f,0.1


End !


In [ ]:
import pandas as pd
data = pd.read_csv('examples/macrodata.csv')
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [ ]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")

periods


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

AttributeError: 'DataFrame' object has no attribute 'year'

In [ ]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

### Pivoting “Wide” to “Long” Format

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [ ]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


## Conclusion

In [2]:
! pip install dataprep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 KB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 23.

In [5]:
from dataprep.datasets import load_dataset
from dataprep.eda import create_report
df = load_dataset("titanic")
create_report(df)

Output hidden; open in https://colab.research.google.com to view.